# 🚀 ALCHEMIST PRODUCTION — Batch Processing

```
╔══════════════════════════════════════════════════════════════════════════════╗
║              FRÉGATE 05_ALCHEMIST — PRODUCTION MODE                          ║
║           Batch Processing • Multi-Scene • Full Post-Production              ║
╚══════════════════════════════════════════════════════════════════════════════╝
```

**Mission**: Traiter toutes les scènes en batch via le pipeline ALCHEMIST complet.

---

## 📦 Setup Production Environment

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# Configuration
DRIVE_ROOT = "/content/drive/MyDrive/DRIVE_EXODUS_V2"
UNIT_ROOT = f"{DRIVE_ROOT}/05_ALCHEMIST_LAB"
CODEBASE = f"{UNIT_ROOT}/CODEBASE"

import sys
sys.path.insert(0, CODEBASE)

print("╔" + "═" * 50 + "╗")
print("║   ALCHEMIST PRODUCTION — EXODUS SYSTEM            ║")
print("╚" + "═" * 50 + "╝")
print(f"\n✓ Environment configured")
print(f"✓ Drive Root: {DRIVE_ROOT}")

In [ ]:
# Install dependencies
!pip install -q numpy scipy Pillow opencv-python-headless imageio tqdm omegaconf

In [ ]:
# Setup Blender 4.0 (si pas déjà sur le Drive)
import os
from pathlib import Path

BLENDER_PATH = f"{DRIVE_ROOT}/EXODUS_AI_MODELS/blender-4.0.0-linux-x64/blender"

if not Path(BLENDER_PATH).exists():
    print("⚠ Blender 4.0 non trouvé sur le Drive")
    print("Téléchargement automatique...")
    
    !mkdir -p {DRIVE_ROOT}/EXODUS_AI_MODELS
    !wget -q https://download.blender.org/release/Blender4.0/blender-4.0.0-linux-x64.tar.xz -O /tmp/blender.tar.xz
    !tar -xf /tmp/blender.tar.xz -C {DRIVE_ROOT}/EXODUS_AI_MODELS/
    !rm /tmp/blender.tar.xz
    
    print("✓ Blender 4.0 installé")
else:
    print(f"✓ Blender 4.0 trouvé: {BLENDER_PATH}")

## 📋 Configuration PRODUCTION_PLAN

In [ ]:
import json

# Chemin vers le PRODUCTION_PLAN.JSON
PRODUCTION_PLAN_PATH = f"{UNIT_ROOT}/IN_RENDER/PRODUCTION_PLAN.JSON"

# Si le fichier n'existe pas, créer un exemple
if not Path(PRODUCTION_PLAN_PATH).exists():
    example_plan = {
        "project": "EXODUS_V2",
        "version": "1.0",
        "scenes": [
            {
                "scene_id": 1,
                "name": "Scene_001_Intro",
                "post_production": {
                    "color_grade": "cinematic_warm",
                    "effects": {
                        "bloom": True,
                        "lens_flare": False,
                        "film_grain": 0.05,
                        "vignette": 0.2
                    },
                    "denoise": True
                }
            },
            {
                "scene_id": 2,
                "name": "Scene_002_Action",
                "post_production": {
                    "color_grade": "neon_nights",
                    "effects": {
                        "bloom": True,
                        "lens_flare": True,
                        "film_grain": 0.08,
                        "vignette": 0.3,
                        "chromatic_aberration": 0.01
                    },
                    "denoise": True
                }
            },
            {
                "scene_id": 3,
                "name": "Scene_003_Dialogue",
                "post_production": {
                    "color_grade": "cinematic_cold",
                    "effects": {
                        "bloom": False,
                        "film_grain": 0.03,
                        "vignette": 0.15
                    },
                    "denoise": True
                }
            }
        ]
    }
    
    with open(PRODUCTION_PLAN_PATH, 'w') as f:
        json.dump(example_plan, f, indent=2)
    print(f"✓ Exemple PRODUCTION_PLAN.JSON créé: {PRODUCTION_PLAN_PATH}")
else:
    print(f"✓ PRODUCTION_PLAN.JSON trouvé")

# Charger et afficher
with open(PRODUCTION_PLAN_PATH, 'r') as f:
    plan = json.load(f)

print(f"\n=== PRODUCTION PLAN ===")
print(f"Project: {plan.get('project', 'N/A')}")
print(f"Scenes: {len(plan.get('scenes', []))}")

for scene in plan.get('scenes', []):
    post = scene.get('post_production', {})
    effects = post.get('effects', {})
    active_fx = [k for k, v in effects.items() if v]
    print(f"\n  Scene {scene['scene_id']}: {scene.get('name', 'Unnamed')}")
    print(f"    Color Grade: {post.get('color_grade', 'natural')}")
    print(f"    Effects: {', '.join(active_fx) if active_fx else 'none'}")
    print(f"    Denoise: {post.get('denoise', True)}")

## 🔍 Validation Pre-Flight

In [ ]:
from pathlib import Path

def validate_setup():
    """Valide que tout est prêt pour la production."""
    errors = []
    warnings = []
    
    # Check Blender
    if not Path(BLENDER_PATH).exists():
        errors.append(f"Blender introuvable: {BLENDER_PATH}")
    
    # Check PRODUCTION_PLAN
    if not Path(PRODUCTION_PLAN_PATH).exists():
        errors.append(f"PRODUCTION_PLAN.JSON introuvable")
    
    # Check EXR files
    render_dir = Path(f"{UNIT_ROOT}/IN_RENDER")
    exr_files = list(render_dir.glob("*.exr"))
    if not exr_files:
        warnings.append("Aucun fichier EXR trouvé dans IN_RENDER/")
    
    # Check LUTs
    luts_dir = Path(f"{UNIT_ROOT}/LUTS")
    lut_files = list(luts_dir.glob("*.cube"))
    if not lut_files:
        warnings.append("Aucun fichier LUT trouvé dans LUTS/")
    
    # Check required LUTs from plan
    required_luts = set()
    for scene in plan.get('scenes', []):
        grade = scene.get('post_production', {}).get('color_grade', '')
        if grade:
            required_luts.add(grade)
    
    available_luts = {f.stem.lower() for f in lut_files}
    missing_luts = required_luts - available_luts
    if missing_luts:
        warnings.append(f"LUTs manquants (presets seront utilisés): {missing_luts}")
    
    return errors, warnings, len(exr_files), len(lut_files)

errors, warnings, exr_count, lut_count = validate_setup()

print("=== Validation Pre-Flight ===")

if errors:
    print("\n✗ ERREURS:")
    for err in errors:
        print(f"   - {err}")

if warnings:
    print("\n⚠ WARNINGS:")
    for warn in warnings:
        print(f"   - {warn}")

print(f"\n✓ Fichiers EXR: {exr_count}")
print(f"✓ Fichiers LUT: {lut_count}")

if not errors:
    print("\n✓ VALIDATION OK — Prêt pour production")
else:
    print("\n✗ VALIDATION ÉCHOUÉE — Corrigez les erreurs")

## 🏭 Exécution Production

In [ ]:
import subprocess
import time
from datetime import datetime

def run_alchemist(scene_id=None, skip_preview=False, verbose=True):
    """Exécute le pipeline ALCHEMIST."""
    start_time = time.time()
    
    cmd = [
        "python", f"{CODEBASE}/EXO_05_ALCHEMIST.py",
        "--drive-root", DRIVE_ROOT,
        "--production-plan", PRODUCTION_PLAN_PATH,
        "--blender-path", BLENDER_PATH
    ]
    
    if scene_id is not None:
        cmd.extend(["--scene", str(scene_id)])
    
    if skip_preview:
        cmd.append("--skip-preview")
    
    if verbose:
        cmd.append("-v")
    
    print(f"\n{'='*60}")
    print(f"ALCHEMIST PRODUCTION")
    print(f"Started: {datetime.now().strftime('%H:%M:%S')}")
    if scene_id:
        print(f"Scene: {scene_id}")
    print(f"{'='*60}")
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    elapsed = time.time() - start_time
    
    print(result.stdout)
    
    if result.returncode == 0:
        print(f"\n✓ PRODUCTION COMPLÉTÉE en {elapsed:.1f}s")
        return True, elapsed
    else:
        print(f"\n✗ PRODUCTION ÉCHOUÉE")
        print(f"STDERR: {result.stderr}")
        return False, elapsed

In [ ]:
# DRY RUN - Validation sans exécution
!python {CODEBASE}/EXO_05_ALCHEMIST.py \
    --drive-root {DRIVE_ROOT} \
    --production-plan {PRODUCTION_PLAN_PATH} \
    --blender-path {BLENDER_PATH} \
    --dry-run -v

In [ ]:
# PRODUCTION COMPLÈTE - Toutes les scènes
# Décommenter pour lancer

# success, elapsed = run_alchemist()
# print(f"\nTemps total: {elapsed:.1f}s")

In [ ]:
# PRODUCTION SCENE UNIQUE
# Utile pour tester une scène spécifique

# SCENE_TO_PROCESS = 1
# success, elapsed = run_alchemist(scene_id=SCENE_TO_PROCESS)

## 📁 Vérification Outputs

In [ ]:
import os
from pathlib import Path

output_dir = Path(f"{UNIT_ROOT}/OUT_GRADED")

print("=== Fichiers Générés ===")
print(f"Dossier: {output_dir}\n")

if output_dir.exists():
    files = sorted(output_dir.glob("*"))
    
    exr_files = [f for f in files if f.suffix == '.exr']
    mp4_files = [f for f in files if f.suffix == '.mp4']
    json_files = [f for f in files if f.suffix == '.json']
    
    print(f"Graded EXR (.exr): {len(exr_files)}")
    for f in exr_files[:10]:
        size_mb = f.stat().st_size / (1024 * 1024)
        print(f"  📦 {f.name} ({size_mb:.2f} MB)")
    if len(exr_files) > 10:
        print(f"  ... et {len(exr_files) - 10} de plus")
    
    print(f"\nPreviews (.mp4): {len(mp4_files)}")
    for f in mp4_files:
        size_mb = f.stat().st_size / (1024 * 1024)
        print(f"  🎬 {f.name} ({size_mb:.2f} MB)")
    
    print(f"\nReports (.json): {len(json_files)}")
    for f in json_files:
        size_kb = f.stat().st_size / 1024
        print(f"  📄 {f.name} ({size_kb:.1f} KB)")
else:
    print("Dossier output non trouvé")

## 📊 Rapport de Production

In [ ]:
import json

report_path = Path(f"{UNIT_ROOT}/OUT_GRADED/alchemist_report.json")

if report_path.exists():
    with open(report_path, 'r') as f:
        report = json.load(f)
    
    print("=== ALCHEMIST REPORT ===")
    print(f"Timestamp: {report.get('timestamp', 'N/A')}")
    print(f"Status: {report.get('status', 'N/A')}")
    
    summary = report.get('summary', {})
    print(f"\nSummary:")
    print(f"  Scenes Total: {summary.get('scenes_total', 0)}")
    print(f"  Scenes Processed: {summary.get('scenes_processed', 0)}")
    print(f"  Scenes Failed: {summary.get('scenes_failed', 0)}")
    print(f"  Total Frames: {summary.get('total_frames', 0)}")
    
    print(f"\nScene Details:")
    for scene in report.get('scenes', []):
        status = "✓" if scene.get('success', False) else "✗"
        print(f"  {status} Scene {scene.get('scene_id', '?')}: {scene.get('frames_input', 0)} frames")
        if scene.get('preview'):
            print(f"      Preview: {scene['preview']}")
else:
    print("Aucun rapport trouvé - exécutez d'abord la production")

## 🎬 Preview des Résultats

In [ ]:
# Afficher les previews MP4 générées
from IPython.display import Video, display

preview_files = list(Path(f"{UNIT_ROOT}/OUT_GRADED").glob("*.mp4"))

if preview_files:
    print(f"=== Previews Disponibles ({len(preview_files)}) ===")
    for preview in preview_files[:3]:  # Limite à 3
        print(f"\n🎬 {preview.name}")
        display(Video(str(preview), width=640))
else:
    print("Aucune preview MP4 générée")

---
## ✅ Production Complete

Les fichiers `.exr` gradés sont prêts pour:
- Export final en ProRes/DNxHR
- Montage dans DaVinci Resolve/Premiere
- Livraison au U06_AIRCRAFT_CARRIER

**Next Step**: Utiliser U06_AIRCRAFT_CARRIER pour encoder les masters finaux.